In [ ]:


import pandas as pd

file_path = "Manufacturing_Line_Productivity.xlsx"
xls = pd.ExcelFile(file_path)

data = {sheet: xls.parse(sheet) for sheet in xls.sheet_names}

In [ ]:
data["Line downtime"] = data["Line downtime"].rename(columns={data["Line downtime"].columns[0]: "Batch"})

In [ ]:
# إزالة القيم الفارغة
data["Line productivity"] = data["Line productivity"].dropna()

# تحويل التواريخ إلى الصيغة الصحيحة
data["Line productivity"]["Date"] = pd.to_datetime(data["Line productivity"]["Date"])

# تحويل الوقت إلى تنسيق مناسب
data["Line productivity"]["Start Time"] = pd.to_datetime(data["Line productivity"]["Start Time"], format="%H:%M:%S").dt.time
data["Line productivity"]["End Time"] = pd.to_datetime(data["Line productivity"]["End Time"], format="%H:%M:%S").dt.time

# التأكد من عدم وجود بيانات مكررة
data["Line productivity"] = data["Line productivity"].drop_duplicates()

# عرض النتيجة بعد التنظيف
print(data["Line productivity"].head())

        Date Product   Batch Operator Start Time  End Time  \
0 2024-08-29  OR-600  422111      Mac   11:50:00  14:05:00   
1 2024-08-29  LE-600  422112      Mac   14:05:00  15:45:00   
2 2024-08-29  LE-600  422113      Mac   15:45:00  17:35:00   
3 2024-08-29  LE-600  422114      Mac   17:35:00  19:15:00   
4 2024-08-29  LE-600  422115  Charlie   19:15:00  20:39:00   

   Minutes Of Production Products.Flavor Products.Size  \
0                  135.0          Orange        600 ml   
1                  100.0      Lemon lime        600 ml   
2                  110.0      Lemon lime        600 ml   
3                  100.0      Lemon lime        600 ml   
4                   84.0      Lemon lime        600 ml   

   Products.Min batch time  
0                       60  
1                       60  
2                       60  
3                       60  
4                       60  


In [ ]:
[data["Line productivity"].info()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Date                     38 non-null     datetime64[ns]
 1   Product                  38 non-null     object        
 2   Batch                    38 non-null     int64         
 3   Operator                 38 non-null     object        
 4   Start Time               38 non-null     object        
 5   End Time                 38 non-null     object        
 6   Minutes Of Production    38 non-null     float64       
 7   Products.Flavor          38 non-null     object        
 8   Products.Size            38 non-null     object        
 9   Products.Min batch time  38 non-null     int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 3.1+ KB


[None]

In [ ]:
data['Line productivity'].drop_duplicates(inplace=True)
data['Line productivity'].duplicated().sum()

0

In [ ]:
data["Products"] = xls.parse("Products")

# التحقق مما إذا كانت الورقة فارغة بعد تحميلها
if data["Products"] is None or data["Products"].empty:
    print("⚠️ ورقة 'Products' لا تحتوي على بيانات!")
else:
    print("✅ تم تحميل ورقة 'Products' بنجاح.")

✅ تم تحميل ورقة 'Products' بنجاح.


In [ ]:
# إزالة القيم الفارغة
data["Products"] = data["Products"].dropna()

# التأكد من عدم وجود بيانات مكررة
data["Products"] = data["Products"].drop_duplicates()

# إزالة المسافات الزائدة من النصوص
data["Products"] = data["Products"].applymap(lambda x: x.strip() if isinstance(x, str) else x)

# عرض النتيجة بعد التنظيف
print(data["Products"].head())

  Product      Flavor    Size  Min batch time
0  OR-600      Orange  600 ml              60
1  LE-600  Lemon lime  600 ml              60
2  CO-600        Cola  600 ml              60
3  DC-600   Diet Cola  600 ml              60
4  RB-600  Root Berry  600 ml              60


<ipython-input-15-392a7f4a7f46>:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data["Products"] = data["Products"].applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [ ]:
# إزالة القيم الفارغة
data["Downtime factors"] = data["Downtime factors"].dropna()

# التأكد من عدم وجود بيانات مكررة
data["Downtime factors"] = data["Downtime factors"].drop_duplicates()

# تحويل القيم النصية إلى Lowercase لتوحيد البيانات
data["Downtime factors"]["Operator Error"] = data["Downtime factors"]["Operator Error"].str.lower()

# عرض النتيجة بعد التنظيف
print(data["Downtime factors"].head())

   Factor         Description Operator Error
0       1      Emergency stop             no
1       2        Batch change            yes
2       3      Labeling error             no
3       4  Inventory shortage             no
4       5       Product spill            yes


In [ ]:
# إعادة تسمية العمود الأول ليكون "Batch"
data["Line downtime"] = data["Line downtime"].rename(columns={data["Line downtime"].columns[0]: "Batch"})

# إزالة الأعمدة غير المسماة (إن وجدت)
data["Line downtime"] = data["Line downtime"].dropna(axis=1, how="all")

# إزالة الصفوف التي تحتوي على قيم فارغة في عمود "Batch"
data["Line downtime"] = data["Line downtime"].dropna(subset=["Batch"])

# التأكد من أن القيم الرقمية ليس بها قيم غير صحيحة أو سلبية
for col in data["Line downtime"].columns[1:]:
    data["Line downtime"][col] = pd.to_numeric(data["Line downtime"][col], errors="coerce")  # تحويل النصوص غير الصالحة إلى NaN
    data["Line downtime"][col] = data["Line downtime"][col].fillna(0)  # استبدال القيم الفارغة بالصفر
    data["Line downtime"][col] = data["Line downtime"][col].apply(lambda x: max(x, 0))  # التأكد من عدم وجود قيم سالبة

# عرض النتيجة بعد التنظيف
print(data["Line downtime"].head())

    Batch  Downtime factor  Downtime minutes  Downtime factors.Description  \
0  422111              2.0                60                           0.0   
1  422111              7.0                15                           0.0   
2  422112              2.0                20                           0.0   
3  422113              2.0                50                           0.0   
4  422117              2.0                10                           0.0   

   Downtime factors.Operator Error  
0                              0.0  
1                              0.0  
2                              0.0  
3                              0.0  
4                              0.0  


In [ ]:
with pd.ExcelWriter("Cleaned_Manufacturing_Data.xlsx") as writer:
    for sheet_name, df in data.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)